<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [8]:
import mediapipe as mp
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
import gc
from operator import add

In [9]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [10]:
cap = cv2.VideoCapture("file://media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/0023159093449660606-CAFETERIA.mp4")
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # face
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Posture
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [11]:
results

mediapipe.python.solution_base.SolutionOutputs

In [12]:
len(results.pose_landmarks.landmark)

33

In [13]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [14]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [15]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [16]:
result_test = extract_keypoints(results)

In [17]:
result_test

array([ 0.53414291,  0.4640485 , -0.59761614, ...,  0.        ,
        0.        ,  0.        ])

In [18]:
def signvideodataframe(filepath):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filepath)

    # Define the path to the videos directory
    path = 'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/'

    # Create a 'Path' column by concatenating the path with 'Video file' column
    df['Path'] = path + df['Video file']
    
#    df['Gloss'] = df['Gloss'].str.replace('\d+', '',regex=True)

    # Calculate the frequency of each gloss and create a 'frequency' column
    df['Frequency'] = df['Gloss'].map(df['Gloss'].value_counts())

    # Sort the DataFrame by the 'Gloss' column
    df = df.sort_values(by='Gloss')

    # Reset the index of the DataFrame
    df = df.reset_index(drop=True)

    return df

In [19]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = round(frames / fps)
    return frames, fps, length

In [20]:
traindf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/train.csv')

In [21]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P31,3827306090663467-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
1,P37,16792698524451422-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
2,P11,6868778695018762-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3,P11,6870709051348651-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4,P50,0719792557216079-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [22]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
40149,P37,9716493262876276-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40150,P31,7550572181460327-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40151,P46,47985881750082227-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40152,P50,04671245574824856-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40153,P51,19959052532136146-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [23]:
traindf['Path'][0]

'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/3827306090663467-1 DOLLAR.mp4'

In [24]:
cap = cv2.VideoCapture(traindf['Path'][0])
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # face
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Posture
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [25]:
traindf.sample(20)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
35587,P11,5142572502501956-THIN.mp4,THIN,C_02_002,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
9572,P27,9025933808422706-DISCONNECT.mp4,DISCONNECT,H_01_017,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
23657,P2,6509144917236493-NUT.mp4,NUT1,D_02_073,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
23264,P27,26718272677630317-NINE.mp4,NINE,E_03_008,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
22628,P16,6123371122658205-MURDER.mp4,MURDER,F_02_077,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
24302,P37,3827863142418859-ORGANIZATION.mp4,ORGANIZATION,J_03_033,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
14719,P37,165101863778635-GLASS 2.mp4,GLASS3,K_01_115,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
22875,P31,8031051338831652-NECK.mp4,NECK,F_02_081,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
2018,P14,29396825939572246-BANANA 2.mp4,BANANA2,K_02_097,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
15883,P50,13742983558106459-HAIRDRYER 3.mp4,HAIRDRYER3,K_01_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14


In [26]:
traindf = traindf[traindf['Frequency'] > 17]
traindf = traindf.reset_index(drop=True)

In [27]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P52,07157565148825373-seedAXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
1,P28,7179300005186042-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
2,P29,16216064841959765-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
3,P37,6193814382865199-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
4,P40,5947453960317015-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19


In [28]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
945,P50,9161417844146778-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
946,P14,00930662603221255-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
947,P27,82063651021682-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
948,P40,5268072837528903-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
949,P40,6363286086951516-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20


In [29]:
traindf.describe()

,Frequency
count,950.000000
mean,19.082105
std,1.316026
min,18.000000
25%,18.000000
50%,19.000000
75%,20.000000
max,24.000000


In [30]:
#traindf['Gloss'] = traindf['Gloss'].str.replace('\d+', '',regex=True)

In [31]:
traindf['Gloss'].nunique()

50

In [32]:
words = traindf['Gloss'].unique()
words

array(['AXE1', 'BACKPACK1', 'BASKETBALL1', 'BEE1', 'BELT1', 'BITE1',
       'BREAKFAST1', 'CANCER1', 'CHRISTMAS1', 'CONFUSED1', 'DARK1',
       'DEAF1', 'DECIDE1', 'DEMAND1', 'DEVELOP1', 'DINNER1', 'DOG1',
       'DOWNSIZE1', 'DRAG1', 'EAT1', 'EDIT1', 'ELEVATOR1', 'FINE1',
       'FLOAT1', 'FOREIGNER1', 'GUESS1', 'HALLOWEEN1', 'HOSPITAL1',
       'HURDLE/TRIP1', 'JEWELRY', 'KNIGHT1', 'LOCK1', 'LUNCH1', 'MAPLE',
       'MEAT1', 'MECHANIC1', 'MICROSCOPE1', 'MOVIE1', 'NOON1', 'PARTY1',
       'PATIENT2', 'RAZOR2', 'RIVER1', 'ROCKINGCHAIR1', 'SHAVE1', 'SINK',
       'SQUEEZE', 'THEY1', 'TWINS1', 'WHATFOR1'], dtype=object)

In [33]:
valdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/val.csv') 

In [34]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P26,22595012150860327-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
1,P39,7421622940519235-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
2,P21,686738356933241-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
3,P12,9219095671540121-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P21,1448188216215387-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4


In [35]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
10299,P21,6959326205750493-ZOOM IN.mp4,ZOOMIN,B_02_056,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
10300,P12,7758716133684984-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10301,P39,844134294032034-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10302,P21,5548062993721732-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10303,P26,19366754134806952-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4


In [36]:
testdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/test.csv')

In [37]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P18,23521769221811684-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
1,P42,023931338852502426-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
2,P49,4893817008748198-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
3,P17,13991818149960333-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
4,P49,34625615110480457-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [38]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
32936,P18,4320702510886756-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32937,P9,7676354653247301-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32938,P47,5386272465310649-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32939,P18,738440364224181-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32940,P17,9953298353288469-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [39]:
valdf = valdf[valdf['Gloss'].isin(words)]
valdf = valdf.reset_index(drop=True)

In [40]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P39,19778675091674147-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
1,P26,8581142177964065-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
2,P39,3877478645046861-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
3,P21,8521417940364975-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK1,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3


In [41]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
186,P26,032677896012150764-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
187,P5,6523145816470133-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
188,P39,0283886564670357-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
189,P21,6142521746642153-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
190,P12,06408604416165864-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5


In [42]:
valdf['Gloss'].nunique()

50

In [43]:
valdf.describe()

,Frequency
count,191.000000
mean,3.963351
std,0.770316
min,3.000000
25%,3.000000
50%,4.000000
75%,4.000000
max,6.000000


In [44]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P39,19778675091674147-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
1,P26,8581142177964065-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
2,P39,3877478645046861-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
3,P21,8521417940364975-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK1,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3


In [45]:
testdf = testdf[testdf['Gloss'].isin(words)]
testdf = testdf.reset_index(drop=True)

In [46]:
testdf['Gloss'].nunique()

50

In [47]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P15,33721516025652254-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
1,P47,519067006979435-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
2,P35,5104381603195376-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3,P42,44458614013793873-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4,P42,9604797909481075-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [48]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
724,P42,3448123355846451-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
725,P47,034046510887022485-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
726,P22,9359050586202402-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
727,P42,2748457214117681-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
728,P15,540014801916062-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18


In [49]:
testdf.describe()

,Frequency
count,729.000000
mean,14.978052
std,2.229192
min,9.000000
25%,14.000000
50%,15.000000
75%,16.000000
max,20.000000


In [50]:
traindf['Gloss'] = traindf['Gloss'].str.replace('\d+', '',regex=True)
valdf['Gloss'] = valdf['Gloss'].str.replace('\d+', '',regex=True)
testdf['Gloss'] = testdf['Gloss'].str.replace('\d+', '',regex=True)

In [51]:
words = traindf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [52]:
words = valdf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [53]:
words = testdf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [54]:
print(traindf['Gloss'].nunique())
print(valdf['Gloss'].nunique())
print(testdf['Gloss'].nunique())

50
50
50


In [59]:
cap = cv2.VideoCapture(traindf.Path[0])
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # rostro
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Mano izquieda (azul)
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Mano derecha (verde)
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Postura
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [56]:
# Define a function to extract keypoints from a single video file
def extract_keypoints_from_file(filepath):
    # Initialize a MediaPipe Holistic model
    holistic = mp.solutions.holistic.Holistic(static_image_mode=False,
                                              model_complexity=2,
                                              min_detection_confidence=0.5,
                                              min_tracking_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(filepath)

    # Initialize an empty list to store the keypoints for each frame
    keypoints = []

    # Loop through the frames of the video
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB color space
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use the Holistic model to detect landmarks for the face, pose, and hands
        results = holistic.process(frame)

        # Extract the keypoints from the results object and append them to the keypoints list
        keypoints.append(extract_keypoints(results))

    # Release the video capture object and the Holistic model
    cap.release()
    holistic.close()

    return np.array(keypoints)

In [57]:
extract_keypoints_from_file(traindf.Path[0]).shape

(100, 1662)

In [60]:
cap = cv2.VideoCapture(traindf.Path[0])
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

100


In [61]:
type(extract_keypoints_from_file(traindf.Path[643]))

numpy.ndarray

In [62]:
traindf.count()

Participant ID    950
Video file        950
Gloss             950
ASL-LEX Code      950
Path              950
Frequency         950
dtype: int64

In [63]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = round(frames / fps)
    return frames, fps, length

In [64]:
frames_from_file(traindf.Path[0])

(100, 31, 3)

In [65]:
tqdm.pandas()
traindf[['Frames', 'FPS', 'Length']] = traindf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|█████████████████████████████████████████| 950/950 [00:56<00:00, 16.84it/s]


In [66]:
traindf['Frames'].max()

540

In [67]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,100,31,3
1,P28,7179300005186042-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,81,30,3
2,P29,16216064841959765-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,47,30,2
3,P37,6193814382865199-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,67,30,2
4,P40,5947453960317015-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,89,30,3


In [68]:
traindf.describe()

,Frequency,Frames,FPS,Length
count,950.000000,950.000000,950.000000,950.000000
mean,19.082105,87.755789,29.354737,3.016842
std,1.316026,46.654745,2.197195,1.568167
min,18.000000,25.000000,11.000000,1.000000
25%,18.000000,60.000000,29.000000,2.000000
50%,19.000000,77.000000,30.000000,3.000000
75%,20.000000,97.000000,30.000000,3.000000
max,24.000000,540.000000,31.000000,18.000000


In [69]:
traindf = traindf[(traindf['Length'] > 1) & (traindf['FPS'] > 28) &(traindf['Frames'] < 131) ]
#filtered_df = traindf[(traindf['Frames'] > 0) & (traindf['Frames'] < 121)]
#filtered_df = traindf[traindf['Frames'] > 0]
traindf= traindf.reset_index(drop=True)
traindf.describe()

,Frequency,Frames,FPS,Length
count,804.000000,804.000000,804.000000,804.000000
mean,19.084577,76.113184,29.680348,2.593284
std,1.305343,20.091965,0.479790,0.671427
min,18.000000,45.000000,29.000000,2.000000
25%,18.000000,60.000000,29.000000,2.000000
50%,19.000000,74.000000,30.000000,2.000000
75%,20.000000,88.000000,30.000000,3.000000
max,24.000000,130.000000,31.000000,4.000000


In [70]:
traindf['Frequency'] = traindf['Gloss'].map(traindf['Gloss'].value_counts())
#traindf = traindf.sort_values(by='Frequency',)

traindf.describe()

,Frequency,Frames,FPS,Length
count,804.000000,804.000000,804.000000,804.000000
mean,16.261194,76.113184,29.680348,2.593284
std,1.722009,20.091965,0.479790,0.671427
min,13.000000,45.000000,29.000000,2.000000
25%,15.000000,60.000000,29.000000,2.000000
50%,16.000000,74.000000,30.000000,2.000000
75%,17.000000,88.000000,30.000000,3.000000
max,20.000000,130.000000,31.000000,4.000000


In [71]:
traindf.head(10)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,100,31,3
1,P28,7179300005186042-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,81,30,3
2,P29,16216064841959765-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,47,30,2
3,P37,6193814382865199-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,67,30,2
4,P40,5947453960317015-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,89,30,3
5,P40,929570016067665-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,112,29,4
6,P16,904767261814883-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,64,30,2
7,P43,016134052760367945-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,75,30,2
8,P40,23099527328070546-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,91,29,3
9,P37,21310388087265242-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,69,30,2


In [72]:
print(traindf['Gloss'].nunique())

50


In [73]:
tqdm.pandas()
valdf[['Frames', 'FPS', 'Length']] = valdf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|█████████████████████████████████████████| 191/191 [00:10<00:00, 18.06it/s]


In [74]:
valdf['Frames'].max()

207

In [75]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P39,19778675091674147-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,121,30,4
1,P26,8581142177964065-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,81,29,3
2,P39,3877478645046861-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,120,30,4
3,P21,8521417940364975-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,71,29,2
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3,93,29,3


In [76]:
valdf.describe()

,Frequency,Frames,FPS,Length
count,191.000000,191.000000,191.000000,191.000000
mean,3.963351,88.905759,29.706806,3.000000
std,0.770316,36.214006,0.456423,1.248157
min,3.000000,37.000000,29.000000,1.000000
25%,3.000000,61.500000,29.000000,2.000000
50%,4.000000,76.000000,30.000000,3.000000
75%,4.000000,117.500000,30.000000,4.000000
max,6.000000,207.000000,30.000000,7.000000


In [77]:
valdf = valdf[(valdf['Length'] > 1) & (valdf['FPS'] > 28) &(valdf['Frames'] < 131) ]
#filtered_df = traindf[(traindf['Frames'] > 0) & (traindf['Frames'] < 121)]
#filtered_df = traindf[traindf['Frames'] > 0]
valdf = valdf.reset_index(drop=True)
valdf.describe()

,Frequency,Frames,FPS,Length
count,162.000000,162.000000,162.000000,162.000000
mean,3.993827,78.203704,29.691358,2.623457
std,0.760001,23.787397,0.463365,0.772311
min,3.000000,45.000000,29.000000,2.000000
25%,3.000000,59.250000,29.000000,2.000000
50%,4.000000,71.500000,30.000000,2.000000
75%,4.000000,92.000000,30.000000,3.000000
max,6.000000,130.000000,30.000000,4.000000


In [78]:
valdf['Frequency'] = valdf['Gloss'].map(valdf['Gloss'].value_counts())
#valdf = valdf.sort_values(by='Frequency',)

valdf.describe()

,Frequency,Frames,FPS,Length
count,162.000000,162.000000,162.000000,162.000000
mean,3.506173,78.203704,29.691358,2.623457
std,0.907162,23.787397,0.463365,0.772311
min,2.000000,45.000000,29.000000,2.000000
25%,3.000000,59.250000,29.000000,2.000000
50%,4.000000,71.500000,30.000000,2.000000
75%,4.000000,92.000000,30.000000,3.000000
max,5.000000,130.000000,30.000000,4.000000


In [79]:
print(valdf['Gloss'].nunique())

50


In [80]:
tqdm.pandas()
testdf[['Frames', 'FPS', 'Length']] = testdf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|█████████████████████████████████████████| 729/729 [00:41<00:00, 17.55it/s]


In [81]:
testdf['Frames'].max()

234

In [82]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P15,33721516025652254-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,196,29,7
1,P47,519067006979435-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,102,26,4
2,P35,5104381603195376-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,70,29,2
3,P42,44458614013793873-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,57,30,2
4,P42,9604797909481075-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,66,30,2


In [83]:
testdf.describe()

,Frequency,Frames,FPS,Length
count,729.000000,729.000000,729.000000,729.000000
mean,14.978052,75.192044,28.696845,2.624143
std,2.229192,27.848864,3.113689,1.003439
min,9.000000,10.000000,14.000000,0.000000
25%,14.000000,56.000000,29.000000,2.000000
50%,15.000000,73.000000,30.000000,3.000000
75%,16.000000,90.000000,30.000000,3.000000
max,20.000000,234.000000,31.000000,8.000000


In [84]:
testdf = testdf[(testdf['Length'] > 1) & (testdf['FPS'] > 28) &(testdf['Frames'] < 131) ]
#filtered_df = traindf[(traindf['Frames'] > 0) & (traindf['Frames'] < 121)]
#filtered_df = traindf[traindf['Frames'] > 0]
testdf = testdf.reset_index(drop=True)
testdf.describe()

,Frequency,Frames,FPS,Length
count,534.000000,534.000000,534.000000,534.000000
mean,15.014981,77.256554,29.702247,2.619850
std,2.217904,19.420007,0.526337,0.670792
min,9.000000,45.000000,29.000000,2.000000
25%,14.000000,61.000000,29.000000,2.000000
50%,15.000000,75.000000,30.000000,3.000000
75%,16.000000,91.000000,30.000000,3.000000
max,20.000000,130.000000,31.000000,4.000000


In [85]:
testdf['Frequency'] = testdf['Gloss'].map(testdf['Gloss'].value_counts())
#testdf = testdf.sort_values(by='Frequency',)

testdf.describe()

,Frequency,Frames,FPS,Length
count,534.000000,534.000000,534.000000,534.000000
mean,11.254682,77.256554,29.702247,2.619850
std,2.311308,19.420007,0.526337,0.670792
min,5.000000,45.000000,29.000000,2.000000
25%,10.000000,61.000000,29.000000,2.000000
50%,12.000000,75.000000,30.000000,3.000000
75%,13.000000,91.000000,30.000000,3.000000
max,15.000000,130.000000,31.000000,4.000000


In [80]:
print(testdf['Gloss'].nunique())

50


In [81]:
keypoints_series = traindf['Path'].progress_apply(extract_keypoints_from_file)

100%|███████████████████████████████████████| 804/804 [2:27:14<00:00, 10.99s/it]


In [110]:
keypoints_series.shape

(804,)

In [84]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_keypoints.npy", keypoints_series)

In [108]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_keypoints.npy",allow_pickle=True)

In [111]:
max(keypoints.shape[0] for keypoints in keypoints_series)

130

In [112]:
min(keypoints.shape[0] for keypoints in keypoints_series)


45

In [113]:
max_len = traindf['Frames'].max()

In [114]:
max_len

130

In [94]:
#keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))
#for i, keypoints in enumerate(keypoints_series):
#    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [115]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [116]:
keypoints_np.shape

(804, 130, 1662)

In [117]:
print(keypoints_np[6])

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.49487716  0.39238185 -1.29218626 ...  0.          0.
   0.        ]
 [ 0.49096876  0.39365393 -1.30090356 ...  0.          0.
   0.        ]
 [ 0.48817223  0.39431143 -1.3236835  ...  0.          0.
   0.        ]]


In [119]:
keypoints_np.shape

(804, 130, 1662)

In [120]:
keypoints_series = valdf['Path'].progress_apply(extract_keypoints_from_file)

100%|█████████████████████████████████████████| 162/162 [29:55<00:00, 11.08s/it]


In [121]:
keypoints_series.shape

(162,)

In [122]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_keypoints.npy", keypoints_series)

In [123]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_keypoints.npy",allow_pickle=True)

In [124]:
keypoints_series.shape

(162,)

In [125]:
#keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))
#for i, keypoints in enumerate(keypoints_series):
#    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [126]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [127]:
keypoints_np.shape

(162, 130, 1662)

In [128]:
keypoints_np.shape

(162, 130, 1662)

In [129]:
keypoints_np

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.5595299 ,  0.3205725 , -1.27106977, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55954963,  0.32069033, -1.25959992, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55960971,  0.3209095 , -1.23169589, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.5183838 ,  0.33223236, -1.15552998, ...,  

In [112]:
keypoints_series = testdf['Path'].progress_apply(extract_keypoints_from_file)

100%|█████████████████████████████████| 705/705 [2:00:53<00:00, 10.29s/it]


In [113]:
keypoints_series.shape

(705,)

In [116]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_keypoints.npy", keypoints_np)

In [ ]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_keypoints.npy", allow_pickle=True)

In [ ]:
keypoints_series.shape

In [ ]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [114]:
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))
for i, keypoints in enumerate(keypoints_series):
    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [115]:
keypoints_np.shape

(705, 130, 1662)

In [118]:
keypoints_np.shape

(705, 130, 1662)

# Preprocessing

In [ ]:
import numpy as np
train_keypoints=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_keypoints.npy",allow_pickle=True)
val_keypoints =np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_keypoints.npy",allow_pickle=True)
test_keypoints=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_keypoints.npy",allow_pickle=True)

print('Training Keypoints Shape:', train_keypoints.shape)
print('Validation Kepoints Shape:', val_keypoints.shape)
print('Test Keypoints Shape:',test_keypoints.shape)

In [ ]:
max_len=130

In [ ]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(train_keypoints), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(train_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
train_keypoints = keypoints_np
print('Training Keypoints Shape:', train_keypoints.shape)

In [ ]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(val_keypoints), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(val_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
val_keypoints = keypoints_np
print('Validation Keypoints Shape:', val_keypoints.shape)

In [ ]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(test_keypoints), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(test_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
test_keypoints = keypoints_np
print('Test Keypoints Shape:', test_keypoints.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from keras.utils import to_categorical
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [6]:
traindf.head()

NameError: name 'traindf' is not defined

In [138]:
encoder = OrdinalEncoder()

In [139]:
traindf['Cat_label'] = encoder.fit_transform(traindf[['Gloss']]).astype(int)
traindf = traindf.sort_values(by='Gloss',)

In [140]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,100,31,3,0
17,P37,29360158406046777-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,73,30,2,0
16,P27,053394218351220823-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,52,29,2,0
15,P40,10598328043940142-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,81,29,3,0
14,P50,3937261764308986-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,72,30,2,0


In [141]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
789,P40,6445331634562388-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,89,30,3,49
788,P37,6237575353180616-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,56,30,2,49
802,P40,5268072837528903-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,92,29,3,49
794,P37,16185522171162914-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,64,30,2,49
803,P40,6363286086951516-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,65,29,2,49


In [142]:
valdf['Cat_label'] = encoder.fit_transform(valdf[['Gloss']]).astype(int)
valdf = valdf.sort_values(by='Gloss',)

In [143]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P39,19778675091674147-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,121,30,4,0
1,P26,8581142177964065-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,81,29,3,0
2,P39,3877478645046861-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,120,30,4,0
3,P21,8521417940364975-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,71,29,2,0
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3,93,29,3,1


In [144]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
159,P39,0283886564670357-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,121,30,4,49
160,P21,6142521746642153-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,57,30,2,49
157,P26,032677896012150764-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,81,29,3,49
158,P5,6523145816470133-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,116,30,4,49
161,P12,06408604416165864-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,71,30,2,49


In [145]:
testdf['Cat_label'] = encoder.fit_transform(testdf[['Gloss']]).astype(int)
testdf = testdf.sort_values(by='Gloss',)

In [146]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P35,5104381603195376-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,70,29,2,0
1,P42,44458614013793873-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,57,30,2,0
2,P42,9604797909481075-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,66,30,2,0
3,P9,30728048195204827-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,96,30,3,0
4,P6,28522130623160047-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,54,29,2,0


In [147]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
522,P6,6734748834983595-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,66,29,2,49
521,P17,33051835760678294-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,52,31,2,49
532,P42,2748457214117681-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,74,30,2,49
526,P42,9684934734400592-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,89,30,3,49
533,P15,540014801916062-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,79,29,3,49


In [148]:
label_map = {label:num for num, label in enumerate(traindf['Gloss'].unique())}

In [149]:
label_map

{'AXE': 0,
 'BACKPACK': 1,
 'BASKETBALL': 2,
 'BEE': 3,
 'BELT': 4,
 'BITE': 5,
 'BREAKFAST': 6,
 'CANCER': 7,
 'CHRISTMAS': 8,
 'CONFUSED': 9,
 'DARK': 10,
 'DEAF': 11,
 'DECIDE': 12,
 'DEMAND': 13,
 'DEVELOP': 14,
 'DINNER': 15,
 'DOG': 16,
 'DOWNSIZE': 17,
 'DRAG': 18,
 'EAT': 19,
 'EDIT': 20,
 'ELEVATOR': 21,
 'FINE': 22,
 'FLOAT': 23,
 'FOREIGNER': 24,
 'GUESS': 25,
 'HALLOWEEN': 26,
 'HOSPITAL': 27,
 'HURDLE/TRIP': 28,
 'JEWELRY': 29,
 'KNIGHT': 30,
 'LOCK': 31,
 'LUNCH': 32,
 'MAPLE': 33,
 'MEAT': 34,
 'MECHANIC': 35,
 'MICROSCOPE': 36,
 'MOVIE': 37,
 'NOON': 38,
 'PARTY': 39,
 'PATIENT': 40,
 'RAZOR': 41,
 'RIVER': 42,
 'ROCKINGCHAIR': 43,
 'SHAVE': 44,
 'SINK': 45,
 'SQUEEZE': 46,
 'THEY': 47,
 'TWINS': 48,
 'WHATFOR': 49}

In [150]:
traindf.sample(10)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
685,P11,8561671103002648-RIVER.mp4,RIVER,A_03_035,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,68,29,2,42
454,P27,9543121609158431-HOSPITAL.mp4,HOSPITAL,B_02_026,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,17,63,29,2,27
618,P37,1012683026825647-NOON.mp4,NOON,G_02_014,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,17,62,30,2,38
673,P36,34327843271669756-RAZOR 2.mp4,RAZOR,K_03_103,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,75,30,2,41
476,P50,22335744767716914-HURDLE-TRIP.mp4,HURDLE/TRIP,G_01_036,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,17,87,30,3,28
482,P23,7185958868464457-JEWELRY.mp4,JEWELRY,H_03_040,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,95,30,3,29
321,P31,6460191084812579-EAT.mp4,EAT,B_02_002,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14,79,30,3,19
307,P37,20831865620547463-DRAG.mp4,DRAG,J_02_039,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,60,29,2,18
735,P37,43245204383901537-SINK.mp4,SINK,E_02_042,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14,71,30,2,45
118,P40,23598266126803202-BREAKFAST.mp4,BREAKFAST,J_01_053,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,85,29,3,6


In [6]:
Ytrain = encoder.fit_transform(traindf[['Gloss']])
Ytest = encoder.fit_transform(testdf[['Gloss']])
Yval = encoder.fit_transform(valdf[['Gloss']])

NameError: name 'encoder' is not defined

In [152]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_labels_array_1D.npy", Ytrain)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_labels_array_1D.npy", Ytest)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_labels_array_1D.npy", Yval)

In [ ]:
Ytrain=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_labels_array_1D.npy")
Ytest=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_labels_array_1D.npy")
Yval=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_labels_array_1D.npy")


In [ ]:
Xtrain = train_keypoints
Xtest = test_keypoints
Xval = val_keypoints

In [ ]:
n_classes = 50
print("Shape before one-hot encoding: ", Ytrain.shape)
Ytrain = to_categorical(Ytrain, n_classes)
print("Shape after one-hot encoding: ", Ytrain.shape)
print("Shape before one-hot encoding: ", Ytest.shape)
Ytest = to_categorical(Ytest, n_classes)
print("Shape after one-hot encoding: ", Ytest.shape)
print("Shape before one-hot encoding: ", Yval.shape)
Yval = to_categorical(Yval, n_classes)
print("Shape after one-hot encoding: ", Yval.shape)

In [ ]:
type(Ytrain)

In [ ]:
Ytrain.shape

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=10)

In [ ]:
Xtrain = np.concatenate((Xtrain, Xtest), axis=0)
print(Xtrain.shape)
Ytrain = np.concatenate((Ytrain, Ytest), axis=0)
print(Ytrain.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler


num_samples, num_frames, num_keypoints = Xtrain.shape
Xtrain = Xtrain.reshape(num_samples, -1)

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
Xtrain = scaler.fit_transform(Xtrain)

# Reshape the scaled data back to the original shape
Xtrain = Xtrain.reshape(num_samples, num_frames, num_keypoints)

In [ ]:
Xtrain.shape

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(input_data_scaled,
                                                    Y,
                                                    test_size=0.,
                                                    random_state=10)

NameError: name 'input_data_scaled' is not defined

In [ ]:
print(Xtrain.shape)
Xtest.shape

In [ ]:
print(Ytrain.shape)
Ytest.shape

In [ ]:
Xtrain

# Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam ,Adagrad, Adadelta, SGD, Lion
import os


In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
tf.random.set_seed(100)

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [ ]:
max_len = 130

In [18]:
model = Sequential(name='Sequential')
act_function = 'tanh'
model.add(LSTM(180, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(LSTM(90, return_sequences=True, activation=act_function))
model.add(LSTM(180, return_sequences=True, activation=act_function))
model.add(LSTM(90, return_sequences=False,activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(45, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(50, activation=act_function))
model.summary()

optimizer = SGD(learning_rate=.00001,nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 64)

2023-08-07 21:31:15.975071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1037 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 130, 180)          1326960   
                                                                 
 lstm_1 (LSTM)               (None, 130, 90)           97560     
                                                                 
 lstm_2 (LSTM)               (None, 130, 180)          195120    
                                                                 
 lstm_3 (LSTM)               (None, 90)                97560     
                                                                 
 dense (Dense)               (None, 90)                8190      
                                                                 
 dense_1 (Dense)             (None, 90)                8190      
                                                                 
 dense_2 (Dense)             (None, 90)                8

2023-08-07 21:31:23.890758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-07 21:31:24.039649: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xa6a3900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-07 21:31:24.039679: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-07 21:31:24.100011: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 [==============================] - 10s 360ms/step - loss: 8.4307 - categorical_accuracy: 0.0171 - val_loss: 5.4406 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
11/11 [==============================] - 2s 218ms/step - loss: 7.6683 - categorical_accuracy: 0.0311 - val_loss: 5.3028 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
11/11 [==============================] - 2s 220ms/step - loss: 7.6718 - categorical_accuracy: 0.0249 - val_loss: 5.0410 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
11/11 [==============================] - 2s 221ms/step - loss: 8.3987 - categorical_accuracy: 0.0187 - val_loss: 5.0319 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
11/11 [==============================] - 2s 221ms/step - loss: 7.4197 - categorical_accuracy: 0.0280 - val_loss: 5.5538 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
11/11 [==============================] - 2s 219ms/step - loss: 8.0407 - categorical_accuracy: 0.0311 - val_loss: 7.9545 - val_categorical_a

KeyboardInterrupt: 

In [18]:
model = Sequential(name='Sequential')
act_function = 'tanh'
model.add(GRU(64, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(GRU(64, return_sequences=False, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(50, activation=act_function))
optimizer = Adagrad(learning_rate=.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 64)
model.summary()

2023-08-07 21:49:05.468776: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1037 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


Epoch 1/1000


2023-08-07 21:49:09.595786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-07 21:49:09.643909: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8864929fe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-07 21:49:09.643936: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-07 21:49:09.647840: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-07 21:49:09.756578: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 [==============================] - 5s 178ms/step - loss: 7.8501 - categorical_accuracy: 0.0311 - val_loss: 8.1508 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
11/11 [==============================] - 1s 82ms/step - loss: 8.1865 - categorical_accuracy: 0.0218 - val_loss: 9.0874 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
11/11 [==============================] - 1s 82ms/step - loss: 7.9489 - categorical_accuracy: 0.0171 - val_loss: 9.2725 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
11/11 [==============================] - 1s 82ms/step - loss: 8.4394 - categorical_accuracy: 0.0264 - val_loss: 7.0158 - val_categorical_accuracy: 0.0435
Epoch 5/1000
11/11 [==============================] - 1s 82ms/step - loss: 7.9150 - categorical_accuracy: 0.0264 - val_loss: 6.5328 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
11/11 [==============================] - 1s 83ms/step - loss: 7.8367 - categorical_accuracy: 0.0249 - val_loss: 7.1868 - val_categorical_accuracy: 0

KeyboardInterrupt: 

In [57]:
model = Sequential()

act_function = 'LeakyReLU'
model.add(LSTM(64, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(GRU(128, return_sequences=True, activation=act_function))
model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(32, activation=act_function))
model.add(Dense(50, activation='softmax'))
optimizer = SGD(learning_rate=.0000005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 32, callbacks=[tb_callback])
model.summary()

Epoch 1/1000
21/21 [==============================] - 16s 566ms/step - loss: 3.9102 - categorical_accuracy: 0.0218 - val_loss: 3.9294 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
21/21 [==============================] - 11s 541ms/step - loss: 3.9102 - categorical_accuracy: 0.0218 - val_loss: 3.9294 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
 7/21 [=========>....................] - ETA: 7s - loss: 3.9032 - categorical_accuracy: 0.0223

KeyboardInterrupt: 

In [ ]:
model = Sequential(name='Sequential') #good, overfitting

act_function = 'LeakyReLU'
model.add(GRU(128, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(GRU(128, return_sequences=False, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(128, activation=act_function, input_shape=(130,1662)))
model.add(Dense(128, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.4, batch_size = 64, callbacks=[tb_callback])
model.summary()

In [46]:
res = model.predict(Xval)

6/6 [==============================] - 1s 50ms/step


In [51]:
np.argmax(res[100])

23

In [52]:
np.argmax(Yval[100])

31

In [43]:
model = Sequential(name='Sequential') ##best

act_function = 'LeakyReLU'
model.add(GRU(32, return_sequences=False, activation=act_function, dropout=.5,input_shape=(130,1662)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(32, activation=act_function))
#model.add(Dense(64, activation=act_function))
model.add(Dense(16, activation=act_function))
model.add(Dropout(.3))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00005,global_clipnorm=.25)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.3, batch_size = 64, callbacks=[tb_callback])
model.summary()

2023-08-08 11:35:54.821749: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1
2023-08-08 11:35:56.029910: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 808928640 exceeds 10% of free system memory.
2023-08-08 11:35:56.734543: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 808928640 exceeds 10% of free system memory.


Epoch 1/1000


2023-08-08 11:35:59.006698: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f82c0387800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-08 11:35:59.006725: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-08 11:35:59.012057: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-08 11:35:59.025994: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-08 11:35:59.088660: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


15/15 [==============================] - 7s 332ms/step - loss: 3.9308 - categorical_accuracy: 0.0224 - val_loss: 3.9131 - val_categorical_accuracy: 0.0249
Epoch 2/1000
15/15 [==============================] - 3s 219ms/step - loss: 3.9224 - categorical_accuracy: 0.0182 - val_loss: 3.9152 - val_categorical_accuracy: 0.0199
Epoch 3/1000
15/15 [==============================] - 3s 223ms/step - loss: 3.9176 - categorical_accuracy: 0.0214 - val_loss: 3.9231 - val_categorical_accuracy: 0.0224
Epoch 4/1000
15/15 [==============================] - 3s 226ms/step - loss: 3.9105 - categorical_accuracy: 0.0288 - val_loss: 3.9270 - val_categorical_accuracy: 0.0249
Epoch 5/1000
15/15 [==============================] - 3s 227ms/step - loss: 3.9105 - categorical_accuracy: 0.0203 - val_loss: 3.9265 - val_categorical_accuracy: 0.0174
Epoch 6/1000
15/15 [==============================] - 3s 231ms/step - loss: 3.9126 - categorical_accuracy: 0.0235 - val_loss: 3.9240 - val_categorical_accuracy: 0.0124
Epoch

15/15 [==============================] - 5s 346ms/step - loss: 3.8267 - categorical_accuracy: 0.0321 - val_loss: 4.1078 - val_categorical_accuracy: 0.0100
Epoch 50/1000
15/15 [==============================] - 4s 270ms/step - loss: 3.8174 - categorical_accuracy: 0.0406 - val_loss: 4.1035 - val_categorical_accuracy: 0.0075
Epoch 51/1000
15/15 [==============================] - 4s 257ms/step - loss: 3.8223 - categorical_accuracy: 0.0438 - val_loss: 4.0769 - val_categorical_accuracy: 0.0100
Epoch 52/1000
15/15 [==============================] - 4s 294ms/step - loss: 3.8173 - categorical_accuracy: 0.0427 - val_loss: 4.1266 - val_categorical_accuracy: 0.0124
Epoch 53/1000
15/15 [==============================] - 5s 351ms/step - loss: 3.8072 - categorical_accuracy: 0.0438 - val_loss: 4.1107 - val_categorical_accuracy: 0.0100
Epoch 54/1000
15/15 [==============================] - 4s 265ms/step - loss: 3.7981 - categorical_accuracy: 0.0630 - val_loss: 4.1235 - val_categorical_accuracy: 0.0100


...

15/15 [==============================] - 4s 274ms/step - loss: 2.7133 - categorical_accuracy: 0.2318 - val_loss: 3.3076 - val_categorical_accuracy: 0.0920
Epoch 194/1000
15/15 [==============================] - 4s 274ms/step - loss: 2.6878 - categorical_accuracy: 0.2415 - val_loss: 3.4190 - val_categorical_accuracy: 0.0746
Epoch 195/1000
15/15 [==============================] - 4s 261ms/step - loss: 2.6622 - categorical_accuracy: 0.2361 - val_loss: 3.3642 - val_categorical_accuracy: 0.0970
Epoch 196/1000
15/15 [==============================] - 4s 261ms/step - loss: 2.6551 - categorical_accuracy: 0.2436 - val_loss: 3.1600 - val_categorical_accuracy: 0.1144
Epoch 197/1000
15/15 [==============================] - 4s 268ms/step - loss: 2.6903 - categorical_accuracy: 0.2382 - val_loss: 3.3657 - val_categorical_accuracy: 0.0871
Epoch 198/1000
15/15 [==============================] - 4s 283ms/step - loss: 2.6859 - categorical_accuracy: 0.2489 - val_loss: 3.2664 - val_categorical_accuracy: 0.

15/15 [==============================] - 4s 265ms/step - loss: 2.4220 - categorical_accuracy: 0.2949 - val_loss: 3.2259 - val_categorical_accuracy: 0.1070
Epoch 242/1000
15/15 [==============================] - 4s 262ms/step - loss: 2.3797 - categorical_accuracy: 0.3024 - val_loss: 3.1776 - val_categorical_accuracy: 0.1169
Epoch 243/1000
15/15 [==============================] - 4s 267ms/step - loss: 2.4060 - categorical_accuracy: 0.3141 - val_loss: 3.2541 - val_categorical_accuracy: 0.1144
Epoch 244/1000
15/15 [==============================] - 4s 265ms/step - loss: 2.4106 - categorical_accuracy: 0.2981 - val_loss: 3.1154 - val_categorical_accuracy: 0.1119
Epoch 245/1000
15/15 [==============================] - 4s 265ms/step - loss: 2.4235 - categorical_accuracy: 0.2842 - val_loss: 3.1320 - val_categorical_accuracy: 0.1095
Epoch 246/1000
15/15 [==============================] - 4s 259ms/step - loss: 2.4151 - categorical_accuracy: 0.2821 - val_loss: 3.2327 - val_categorical_accuracy: 0.

15/15 [==============================] - 4s 270ms/step - loss: 2.2065 - categorical_accuracy: 0.3344 - val_loss: 3.2616 - val_categorical_accuracy: 0.1194
Epoch 290/1000
15/15 [==============================] - 4s 266ms/step - loss: 2.2188 - categorical_accuracy: 0.3194 - val_loss: 3.1444 - val_categorical_accuracy: 0.1294
Epoch 291/1000
15/15 [==============================] - 4s 263ms/step - loss: 2.2085 - categorical_accuracy: 0.3397 - val_loss: 3.1506 - val_categorical_accuracy: 0.1244
Epoch 292/1000
15/15 [==============================] - 4s 262ms/step - loss: 2.2201 - categorical_accuracy: 0.3451 - val_loss: 3.1253 - val_categorical_accuracy: 0.1194
Epoch 293/1000
15/15 [==============================] - 4s 266ms/step - loss: 2.2413 - categorical_accuracy: 0.3162 - val_loss: 3.1109 - val_categorical_accuracy: 0.1219
Epoch 294/1000
15/15 [==============================] - 4s 262ms/step - loss: 2.1501 - categorical_accuracy: 0.3579 - val_loss: 3.1646 - val_categorical_accuracy: 0.

15/15 [==============================] - 4s 262ms/step - loss: 2.0359 - categorical_accuracy: 0.3974 - val_loss: 3.2861 - val_categorical_accuracy: 0.1194
Epoch 338/1000
15/15 [==============================] - 4s 260ms/step - loss: 2.0179 - categorical_accuracy: 0.3729 - val_loss: 3.3326 - val_categorical_accuracy: 0.1244
Epoch 339/1000
15/15 [==============================] - 4s 263ms/step - loss: 1.9970 - categorical_accuracy: 0.3846 - val_loss: 3.1749 - val_categorical_accuracy: 0.1343
Epoch 340/1000
15/15 [==============================] - 4s 261ms/step - loss: 2.0076 - categorical_accuracy: 0.3835 - val_loss: 3.1767 - val_categorical_accuracy: 0.1244
Epoch 341/1000
15/15 [==============================] - 4s 259ms/step - loss: 2.0389 - categorical_accuracy: 0.3835 - val_loss: 3.0772 - val_categorical_accuracy: 0.1368
Epoch 342/1000
15/15 [==============================] - 4s 265ms/step - loss: 1.9971 - categorical_accuracy: 0.3857 - val_loss: 3.2752 - val_categorical_accuracy: 0.

KeyboardInterrupt: 

In [43]:
model = Sequential(name='Sequential')

act_function = 'selu'
model.add(GRU(64, return_sequences=False, activation=act_function, dropout=.5,kernel_initializer='he_uniform',input_shape=(130,1662)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(32, activation=act_function))
#model.add(Dense(64, activation=act_function))
model.add(Dense(16, activation=act_function))
model.add(Dropout(.3))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.005,global_clipnorm=10)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=5000, validation_split=.4, batch_size = 64, callbacks=[tb_callback])
model.summary()

2023-08-08 17:36:17.297939: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1
2023-08-08 17:36:18.593263: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1040544960 exceeds 10% of free system memory.


Epoch 1/5000


2023-08-08 17:36:21.722239: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7b18452100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-08 17:36:21.722274: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-08 17:36:21.728081: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-08 17:36:21.741053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-08 17:36:21.805903: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


19/19 [==============================] - 7s 234ms/step - loss: 4.0885 - categorical_accuracy: 0.0291 - val_loss: 4.1440 - val_categorical_accuracy: 0.0000e+00
Epoch 2/5000
19/19 [==============================] - 4s 217ms/step - loss: 4.0645 - categorical_accuracy: 0.0191 - val_loss: 4.0806 - val_categorical_accuracy: 0.0000e+00
Epoch 3/5000
19/19 [==============================] - 4s 225ms/step - loss: 3.9947 - categorical_accuracy: 0.0183 - val_loss: 4.0219 - val_categorical_accuracy: 0.0299
Epoch 4/5000
19/19 [==============================] - 4s 221ms/step - loss: 3.9759 - categorical_accuracy: 0.0183 - val_loss: 3.9075 - val_categorical_accuracy: 0.0149
Epoch 5/5000
19/19 [==============================] - 4s 224ms/step - loss: 3.9454 - categorical_accuracy: 0.0191 - val_loss: 3.9256 - val_categorical_accuracy: 0.0075
Epoch 6/5000
19/19 [==============================] - 4s 226ms/step - loss: 3.9341 - categorical_accuracy: 0.0282 - val_loss: 3.9267 - val_categorical_accuracy: 0.00

19/19 [==============================] - 5s 244ms/step - loss: 3.5412 - categorical_accuracy: 0.0822 - val_loss: 4.0214 - val_categorical_accuracy: 0.0000e+00
Epoch 50/5000
19/19 [==============================] - 5s 244ms/step - loss: 3.5468 - categorical_accuracy: 0.0831 - val_loss: 3.9519 - val_categorical_accuracy: 0.0000e+00
Epoch 51/5000
19/19 [==============================] - 5s 244ms/step - loss: 3.5162 - categorical_accuracy: 0.0872 - val_loss: 3.9390 - val_categorical_accuracy: 0.0075
Epoch 52/5000
19/19 [==============================] - 5s 246ms/step - loss: 3.5342 - categorical_accuracy: 0.0955 - val_loss: 3.8922 - val_categorical_accuracy: 0.0149
Epoch 53/5000
19/19 [==============================] - 5s 250ms/step - loss: 3.4977 - categorical_accuracy: 0.1030 - val_loss: 3.9604 - val_categorical_accuracy: 0.0000e+00
Epoch 54/5000
19/19 [==============================] - 5s 245ms/step - loss: 3.4793 - categorical_accuracy: 0.1071 - val_loss: 3.9357 - val_categorical_accur

KeyboardInterrupt: 

In [43]:
model = Sequential(name='Sequential') 

act_function = 'LeakyReLU'
initializer = 'lecun_uniform'
model.add(GRU(32, return_sequences=True, kernel_initializer=initializer, activation=act_function, dropout=0,input_shape=(130,1662)))
model.add(GRU(8, return_sequences=False, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(32, activation=act_function, kernel_initializer=initializer))
model.add(Dense(32, activation=act_function, kernel_initializer=initializer))
model.add(Dense(16, activation=act_function, kernel_initializer=initializer))
#model.add(Dropout(.3))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.3, batch_size = 64, callbacks=[tb_callback])
model.summary()

2023-08-08 19:25:08.494048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


2023-08-08 19:25:09.673381: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 808928640 exceeds 10% of free system memory.
2023-08-08 19:25:10.394759: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 808928640 exceeds 10% of free system memory.


Epoch 1/1000


2023-08-08 19:25:13.937670: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f402c03b3b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-08 19:25:13.937718: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-08 19:25:13.943961: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-08 19:25:13.959021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-08 19:25:14.021873: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


15/15 [==============================] - 10s 456ms/step - loss: 3.9890 - categorical_accuracy: 0.0224 - val_loss: 4.0937 - val_categorical_accuracy: 0.0025
Epoch 2/1000
15/15 [==============================] - 6s 382ms/step - loss: 3.9361 - categorical_accuracy: 0.0310 - val_loss: 4.1797 - val_categorical_accuracy: 0.0124
Epoch 3/1000
15/15 [==============================] - 6s 398ms/step - loss: 3.9642 - categorical_accuracy: 0.0331 - val_loss: 4.0007 - val_categorical_accuracy: 0.0075
Epoch 4/1000
15/15 [==============================] - 6s 414ms/step - loss: 3.9640 - categorical_accuracy: 0.0182 - val_loss: 4.0517 - val_categorical_accuracy: 0.0100
Epoch 5/1000
15/15 [==============================] - 6s 410ms/step - loss: 3.9601 - categorical_accuracy: 0.0267 - val_loss: 4.1521 - val_categorical_accuracy: 0.0274
Epoch 6/1000
15/15 [==============================] - 6s 418ms/step - loss: 3.9760 - categorical_accuracy: 0.0278 - val_loss: 4.3156 - val_categorical_accuracy: 0.0274
Epoc

KeyboardInterrupt: 

In [44]:
initializer

NameError: name 'initializer' is not defined

In [45]:
model = Sequential(name='Sequential')

act_function = 'LeakyReLU'
model.add(GRU(128, return_sequences=False, activation=act_function, input_shape=(130,1662)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(128, activation=act_function, input_shape=(130,1662)))
model.add(Dense(128, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dropout(.2))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 32, callbacks=[tb_callback])
model.summary()

2023-08-08 10:10:09.127784: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 924736800 exceeds 10% of free system memory.
2023-08-08 10:10:09.961623: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 924736800 exceeds 10% of free system memory.


Epoch 1/1000
34/34 [==============================] - 9s 207ms/step - loss: 3.9179 - categorical_accuracy: 0.0224 - val_loss: 4.3422 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
34/34 [==============================] - 7s 201ms/step - loss: 3.8832 - categorical_accuracy: 0.0234 - val_loss: 5.0312 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
34/34 [==============================] - 7s 192ms/step - loss: 3.8310 - categorical_accuracy: 0.0215 - val_loss: 5.5114 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
34/34 [==============================] - 7s 202ms/step - loss: 3.8174 - categorical_accuracy: 0.0271 - val_loss: 5.8857 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
34/34 [==============================] - 7s 210ms/step - loss: 3.7950 - categorical_accuracy: 0.0336 - val_loss: 5.7742 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
34/34 [==============================] - 7s 212ms/step - loss: 3.7763 - categorical_accuracy: 0.0262 - val_loss: 6.2825 - val_c

KeyboardInterrupt: 